<a href="https://colab.research.google.com/github/NeetishPathak/colab-notebooks/blob/main/final_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install requests crewai crewai[tools] openai termcolor

In [ ]:
!wget https://raw.githubusercontent.com/NeetishPathak/dbops-agentic-crew/main/app/resources/pdfs/Redis.pdf -O Redis.pdf

In [ ]:
import os
from google.colab import userdata
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

### GCP Setup for Agents to use later

In [ ]:
## We use GCP to setup a vm . Some presteps to setup a GCP project resources

# Install the CLI
!apt-get install -y lsb-release
!curl -s https://packages.cloud.google.com/apt/doc/apt-key.gpg | apt-key add -
!echo "deb http://packages.cloud.google.com/apt cloud-sdk main" | tee -a /etc/apt/sources.list.d/google-cloud-sdk.list
!apt-get update -q
!apt-get install -y google-cloud-sdk


In [ ]:
## Setup terraform
!wget -O - https://apt.releases.hashicorp.com/gpg | gpg --dearmor -o /usr/share/keyrings/hashicorp-archive-keyring.gpg
!echo "deb [arch=$(dpkg --print-architecture) signed-by=/usr/share/keyrings/hashicorp-archive-keyring.gpg] https://apt.releases.hashicorp.com $(grep -oP '(?<=UBUNTU_CODENAME=).*' /etc/os-release || lsb_release -cs) main" | tee /etc/apt/sources.list.d/hashicorp.list
!apt update && apt install terraform
!terraform version

In [ ]:
# Authenticate the user to use GCP
from google.colab import auth
auth.authenticate_user()

In [ ]:
my_gcp_project_name = userdata.get('MY_GCP_PROJECT')
!gcloud config set project {my_gcp_project_name}

!gcloud services enable compute.googleapis.com

In [ ]:
!gcloud iam service-accounts create terraform-agent --display-name="Terraform Agent"

for role in [
    "roles/compute.admin",
    "roles/iam.serviceAccountUser",
    "roles/serviceusage.serviceUsageAdmin"
]:
    !gcloud projects add-iam-policy-binding {my_gcp_project_name} \
        --member="serviceAccount:terraform-agent@{my_gcp_project_name}.iam.gserviceaccount.com" \
        --role="{role}" --quiet

!gcloud iam service-accounts keys create terraform-agent-key.json \
    --iam-account=terraform-agent@{my_gcp_project_name}.iam.gserviceaccount.com

In [ ]:
# Terraform Config + CrewAI Tooling: Provision GCP VM with Docker Installed (with Colab Secrets Support)

# --- Terraform Config (main.tf) ---
main_tf = """
provider "google" {
  project     = var.project
  region      = var.region
  zone        = var.zone
  credentials = file(var.credentials_file)
}

resource "google_compute_instance" "docker_vm" {
  name         = var.instance_name
  machine_type = var.machine_type
  zone         = var.zone

  boot_disk {
    initialize_params {
      image = "ubuntu-os-cloud/ubuntu-2204-lts"
      size  = 30
    }
  }

  network_interface {
    network = "default"
    access_config {}
  }

  metadata_startup_script = <<-EOT
    #!/bin/bash
    curl -fsSL https://get.docker.com | sh
  EOT
}
"""

# --- Terraform Variables (variables.tf) ---
variables_tf = """
variable "project" {}
variable "region" { default = "us-central1" }
variable "zone" { default = "us-central1-a" }
variable "credentials_file" { default = "account.json" }
variable "instance_name" { default = "docker-agent-vm" }
variable "machine_type" { default = "e2-micro" }
"""

# --- CrewAI Tool: Terraform VM Creator (Colab Secrets) ---
import subprocess
import os, time
from crewai.tools import BaseTool

class GCPDockerVMTool(BaseTool):
    def __init__(self, name="GCPDockerVMProvisioner", description="Creates a GCP VM using Terraform with Docker pre-installed, loading service account from Colab secrets.", **kwargs):
        super().__init__(name=name, description=description, **kwargs)

    def _run(self, input_type: dict) -> str:
        try:
            from google.colab import userdata  # Only available in Colab
            tf_path="terraform"

            if os.path.exists(tf_path):
              shutil.rmtree(tf_path)

            os.makedirs("terraform", exist_ok=True)

            # Write Terraform files
            with open(os.path.join(tf_path, "main.tf"), "w") as f:
                f.write(main_tf)
            with open(os.path.join(tf_path, "variables.tf"), "w") as f:
                f.write(variables_tf)

            project_id = userdata.get("MY_GCP_PROJECT")

            if not project_id:
                return "❌ Missing required secrets: MY_GCP_PROJECT"

            # Load service account secrets from
            key_file_path = "terraform-agent-key.json"

            if not os.path.exists(key_file_path):
                return f"❌ Key file '{key_file_path}' does not exist."

            with open(key_file_path, "r") as f:
                key_data = f.read().strip()

            if not key_data or not key_data.startswith('{'):
                return "❌ Key file is empty or not a valid JSON key."

            credentials_path = os.path.join(tf_path, "account.json")
            with open(credentials_path, "w") as f:
                f.write(key_data)

            # Create terraform.tfvars
            with open(os.path.join(tf_path, "terraform.tfvars"), "w") as f:
                f.write(f"""
project = "{project_id}"
region = "us-central1"
zone = "us-central1-a"
credentials_file = "account.json"
""")
            apply_init = subprocess.run(["terraform", "init"], cwd=tf_path, check=True, capture_output=True, text=True)
            if apply_init.returncode != 0:
                return f"❌ Terraform init failed:\nSTDOUT:\n{apply_init.stdout}\nSTDERR:\n{apply_init.stderr}"

            apply_proc = subprocess.run(["terraform", "apply", "-auto-approve"], cwd=tf_path, check=True, capture_output=True, text=True)
            if apply_proc.returncode != 0:
                if "Error " in apply_proc.stderr and "already exists" in apply_proc.stderr:
                    return "⚠️ VM already exists — skipping create."
                return f"❌ Terraform apply failed:\nSTDOUT:\n{apply_proc.stdout}\nSTDERR:\n{apply_proc.stderr}"

            # Optional: Check if Docker is working
            # check_docker = subprocess.run(
            #     ["gcloud", "compute", "ssh", "docker-agent-vm", "--zone", "us-central1-a",
            #      "--command", "docker --version"],
            #     capture_output=True, text=True
            # )
            # if check_docker.returncode != 0:
            #     return f"⚠️ VM created, but Docker check failed:\n{check_docker.stderr}"

            return f"✅ GCP VM created with Docker installed.\n"


        except subprocess.CalledProcessError as e:
            return f"❌ Terraform failed: {e}"
        except Exception as e:
            return f"❌ Unexpected error: {str(e)}"

gcp_docker_vm_tool = GCPDockerVMTool(description="Deploys a VM in GCP with docker installed.")


In [ ]:
# Crew AI Tools

from crewai_tools import FileWriterTool, PDFSearchTool

from pathlib import Path
import os
import platform
import subprocess
import shutil
import urllib.request

# File Write Tools
file_writer_tool = FileWriterTool()

# PDF Search Tool
pdf_file = "Redis.pdf"
pdf_rag_search_tool = PDFSearchTool(pdf_file)




In [ ]:
# Custom CrewAI Tools
from crewai.tools import BaseTool

## Kind Installer Tool
# Kind is a k8s installer for a node
class KindInstaller(BaseTool):
  def __init__(self, name="KindInstaller", description="Installs Kind...", **kwargs):
        super().__init__(name=name, description=description, **kwargs)

  def _run(self, input_type: dict) -> str:
    try:

      if shutil.which("kind"):
        return "✅ 'kind' is already installed."

      system = platform.system().lower()

      if system not in ["linux", "darwin"]:
        return f"❌ Unsupported OS: {system}. Manual install required."

      # Download the latest Kind binary
      arch = subprocess.check_output(["uname", "-m"]).decode().strip()
      arch_map = {
        "linux": {"x86_64": "amd64", "aarch64": "arm64"},
        "darwin": {"x86_64": "amd64", "arm64": "arm64"}
      }
      arch_name = arch_map.get(system, {}).get(arch)
      if not arch_name:
        return f"❌ Unsupported architecture: {arch} on {system}."
      url = f"https://kind.sigs.k8s.io/dl/v0.29.0/kind-{system}-{arch_name}"

      tmp_kind_file =  os.path.join(os.getcwd(), "kind")
      urllib.request.urlretrieve(url, tmp_kind_file)

      # Make executable
      subprocess.run(["chmod", "+x", tmp_kind_file], check=True)
      # Move to a directory in PATH
      os.environ["PATH"] = f"{os.getcwd()}:{os.environ['PATH']}"

      return "✅ Kind has been installed successfully."

    except Exception as e:
      return f"❌ Failed to install Kind: {e}"

  def _arun(self, input: str = "") -> str:
      raise NotImplementedError("Async version not implemented.")


kind_cluster_tool = KindInstaller(description="Installs a Kind Tool for Kubernetes deployments.")


In [ ]:
# CrewAI Agents

from crewai import Agent

## Agents
architect = Agent(
  role="{data_store} System Architect",
  goal="Design optimal {data_store} cluster configurations tailored to user workloads and environments.",
  backstory=("You are a principal system architect specializing in distributed data stores, particularly {data_store}. "
        "You are an expert in {data_store} internals and Kubernetes-ready deployments."
        "Use the tool to get the guidance on redis configuration. Do not use external knowledge beyond what is contained in the pdf file {pdf_file} "
        "Pass your question as a plain string to the 'query' parameter — for example: "
        "{ \"query\": \"optimal Redis config for 100 ops/sec workload\" }. "
        "Do not include 'description' or 'type' keys. Do not wrap the string inside another dictionary."
        "Just say, your capacity is limited to certain use cases only at the moment if the query is outside  the scope of the pdf."),
  verbose=False,
  tools=[pdf_rag_search_tool],
  allow_delegation=False,
)


manifest_writer_agent = Agent(
  role="{data_store} manifest Writer",
  goal="Vets and writes yaml configurations to the target directory.",
  backstory=("You are a dedicated writer agent specializing in {data_store} configurations. "
        "You leverage the File Writer Tool to create and update configuration files as needed."),
  verbose=False,
  tools=[file_writer_tool],
  allow_delegation=False,
)

infra_docker_vm_agent = Agent(
    role="Infrastructure Automator",
    goal="Provision a GCP VM with Docker installed using Terraform.",
    backstory=("You are an experienced infra admin responsible for setting up vm on Google Cloud Platform using terraforms"
              "You ensure the vm is properly setup, docker is installed which is important for kind k8s cluster to be installed"
              "When using the tool, just pass the terraform_directory_path i.e. {terraform_dir} as a string that you have received as input parameter "),
    verbose=False,
    tools=[gcp_docker_vm_tool],
    allow_delegation=False
)


infra_k8s_kind_agent = Agent(
  role="Infra Admin managing kind cluster",
  goal = "Manage the kind cluster setup and ensure the environment is ready for deployment.",
  backstory=("You are an experienced infra admin responsible for setting up and managing the kind cluster. "
        "You ensure that the environment is properly configured and ready for deployment of {data_store} configurations."),
  verbose=False,
  tools=[kind_cluster_tool],
  allow_delegation=False
)




In [ ]:
from crewai import Task

## Tasks

# ----------------------------------------------------
# Define the architect's task to suggest a cluster design
# ----------------------------------------------------
design_cluster = Task(
    description=(
        "Design an optimal {data_store} architecture for this workload:\n"
        "{workload_description}\n\n"
        "Respond in this exact format:\n"
        "---\n"
        "Architecture Summary:\n<brief text>\n\n"
        "---\n"
        "Key Decisions:\n- <bullet1>\n- <bullet2>\n\n"
        "```yaml\n<valid Kubernetes YAML>\n```\n"
        "Do not add any extra commentary."
    ),
    expected_output="A summary, key decisions, and a valid Kubernetes YAML block.",
    agent=architect,
)

# ----------------------------------------------------
# Define the writer's task to create a manifest writer
# ----------------------------------------------------
write_manifest = Task(
    description=(
        "Validate the Kubernetes YAML and write it to `{target_directory}/{data_store}.yaml`.\n"
        "Use the File Writer Tool.\n"
        "If the file exists, overwrite it.\n"
        "**Important**: The YAML must be valid and ready for `kubectl apply`."
    ),
    expected_output="Confirmation that the YAML was successfully written.",
    agent=manifest_writer_agent,
    context=[design_cluster]
)

# ----------------------------------------------------
# Define the Infra's setup task to create a docker vm agent
# ----------------------------------------------------
infra_vm_setup = Task(
    description=(
        "Set up a vm on gcp \n"
        "Use the infra_docker_vm_agent Tool. Pass a directory as a string to store terraform configurations\n"
        "If already set up, confirm readiness. Check docker is ready after installation is complete\n"
        "Provide access instructions if setup is successful."
    ),
    expected_output="Confirmation that the vm with docker is ready for k8s deployment.",
    agent=infra_docker_vm_agent,
)


# ----------------------------------------------------
# Define the Infra's setup task to create a kind k8s cluster
# ----------------------------------------------------
infra_k8s_setup = Task(
    description=(
        "Set up and validate a kind cluster for deploying {data_store}.\n"
        "Use the Kind Installer Tool.\n"
        "If already set up, confirm readiness.\n"
        "Provide access instructions if setup is successful."
    ),
    expected_output="Confirmation that the kind cluster is ready for deployment.",
    agent=infra_k8s_kind_agent,
)

In [ ]:
from crewai import Crew, Process


## Crew

crew = Crew(
  # agents=[architect, manifest_writer_agent, infra_docker_vm_agent, infra_k8s_kind_agent],
  agents=[infra_docker_vm_agent],
  # tasks=[design_cluster, write_manifest, infra_vm_setup, infra_k8s_setup],
  tasks=[infra_vm_setup],
  process=Process.sequential,
  verbose=True
)

In [ ]:
import os
from pathlib import Path
from termcolor import colored
import warnings

warnings.filterwarnings("ignore", category=UserWarning, module="pydantic")

inputs = {
  "data_store": "redis",
  "workload_description": (
    "A low-volume website. "
    "It handles 100 ops/sec with 80% reads, 20% writes. "
  ),
  "target_directory": "k8s_configs",
  "target_file": "redis.yaml",
  "pdf_file": "Redis.pdf",
  "terraform_dir" : "terraform"
}

def run_sync_crew():
    """
    Kick off the crew process with the provided inputs.
    """
    crew.kickoff(inputs=inputs)


print(colored("🚀 Crew execution started...\n", "yellow"))

run_sync_crew()

print(colored("\n🧠 All Tasks Finished  ", "green"))

## Validation Steps

In [ ]:
!gcloud compute ssh docker-agent-vm --zone us-central1-a --command "uname -a"

In [ ]:
!gcloud compute ssh docker-agent-vm --zone us-central1-a --command "docker --version"

In [ ]:
!gcloud compute ssh docker-agent-vm --zone us-central1-a --command "hostname; docker ps "

In [ ]:
!gcloud compute ssh docker-agent-vm --zone us-central1-a --command "docker run -d -p 6379:6379 --name redis redis"

In [ ]:
!gcloud compute ssh docker-agent-vm --zone us-central1-a --command "hostname; docker ps "

In [ ]:
!gcloud compute instances delete docker-agent-vm